In [33]:
# No warnings
import warnings
warnings.filterwarnings('ignore') # Filter out warnings

# data analysis and wrangling
import pandas as pd
import numpy as np
import random as rnd

pd.set_option('display.max_columns', 100) # Print 100 Pandas columns

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB # Gaussian Naive Bays
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier #stochastic gradient descent
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import xgboost as xgb

import tensorflow as tf
from datetime import datetime
import os
import pathlib
from pathlib import Path

In [34]:
h_df = pd.read_csv('HainanClean.csv')
columns = h_df.columns.values
columns_out = columns[5:12]
columns_in = columns[12 :]

In [35]:
h_df_norm = pd.read_csv('HainanClean.csv')
for i in columns_in:
    h_df_norm[i] = pd.to_numeric(h_df_norm[i], errors='coerce')
    h_df_norm[i] = (h_df_norm[i]-h_df_norm[i].min())/(h_df_norm[i].max() - h_df_norm[i].min()) 

for i in columns:
    h_df_norm[i] = h_df_norm[i].fillna(float(0))
    
h_df_norm.head()

,Year,Month,Day,Month #,Day #,Raw Biogas Produced (m3),BioCNG Produced （m3),BioCNG Sold (m3),Vehicle use (m3),Liquid Fertilizer Produced (t),Solid fertilizer produced (t),Wastewater flow to WWTP (unit?),Solid residues (kg),Pig Manure (t),Cassava (t),Fish waste water (t),Kitchen food waste (t),Municipal fecal residue (t),Tea waste (t),Chicken litter (t),Bagasse feed (t),Alcohol waste (t),Chinese medicine waste (t),Energy grass (t),Banana fruit shafts (t),Lemon waste (t),Percolate (t),Other waste (t),50% NaOH/kg,FeCl2/kg,PAM/kg,Defoamer/kg,Project electricity use/kWh,Office space electricity use/kWh,Water/m3,Diesel/L
0,2014,April,1,0.0,0.0,5154.0,1710.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2014,April,2,0.0,0.0,3426.0,1529.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2014,April,3,0.0,0.0,5885.0,1795.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2014,April,4,0.0,0.0,3727.0,1361.0,0.0,0.0,0.0,0.0,0.0,0.0,0.002185,0.0,0.0,0.0,0.0,0.0,0.0,0.152851,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2014,April,5,0.0,0.0,4072.0,2436.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.188125,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [36]:
columns_out
X_1 = h_df_norm.drop(columns_out, axis=1) # Training & Validation data
X = X_1.drop(['Solid residues (kg)', 'Year', 'Month', 'Day', 'Month #', 'Day #', 'Project electricity use/kWh', 'Office space electricity use/kWh', 'Diesel/L', 'Defoamer/kg', 'Water/m3'], axis=1)
Y = h_df['Raw Biogas Produced (m3)'] # Response / Target Variable
#np.random.seed(1337)
#X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.2)
length = h_df_norm['Pig Manure (t)'].count()
cut = int(round(length*0.8))
X_train = X[0 : cut]
Y_train = Y[0 : cut]
X_val = X[cut : length]
Y_val = Y[cut : length]

In [37]:
X.head()

,Pig Manure (t),Cassava (t),Fish waste water (t),Kitchen food waste (t),Municipal fecal residue (t),Tea waste (t),Chicken litter (t),Bagasse feed (t),Alcohol waste (t),Chinese medicine waste (t),Energy grass (t),Banana fruit shafts (t),Lemon waste (t),Percolate (t),Other waste (t),50% NaOH/kg,FeCl2/kg,PAM/kg
0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.002185,0.0,0.0,0.0,0.0,0.0,0.0,0.152851,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.188125,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [38]:
Y.head()

0    5154.0
1    3426.0
2    5885.0
3    3727.0
4    4072.0
Name: Raw Biogas Produced (m3), dtype: float64

In [57]:
linreg = LinearRegression()
linreg.fit(X_train, Y_train) # fit
acc = linreg.score(X_val, Y_val)
print('coefficients:', linreg.coef_)
print('r squared:', acc)

coefficients: [  3.29822303e-01   3.95157240e+00   4.99056588e-01   7.90866664e-01
   4.18961144e+00  -3.80314837e-01   1.81258997e+00   2.05928876e+00
   5.05323612e-01   5.08327721e-01   3.25913539e-01   2.66453526e-15
   1.77635684e-15   2.35530058e+00   9.31018678e-01  -5.14296457e-01
   6.95552583e-01   5.00808364e-01]
r squared: -0.46980385458


In [50]:
X_1 = h_df_norm.drop(columns_out, axis=1) # Training & Validation data
X = X_1.drop(['Solid residues (kg)', 'Year', 'Month', 'Day', 'Month #', 'Day #', 'Project electricity use/kWh', 'Office space electricity use/kWh', 'Diesel/L', 'Defoamer/kg', 'Water/m3'], axis=1)
Y_1 = h_df['Raw Biogas Produced (m3)'] # Response / Target Variable
bins = 5
labels = list(range(1,bins+1))
Y = pd.cut(Y_1, bins = bins, labels = labels)

#np.random.seed(1337)
#X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.2)
length = h_df_norm['Pig Manure (t)'].count()
cut = int(round(length*0.8))
X_train = X[0 : cut]
Y_train = Y[0 : cut]
X_val = X[cut : length]
Y_val = Y[cut : length]

In [51]:
logreg = LogisticRegression() # instantiate
logreg.fit(X_train, Y_train) # fit
Y_pred = logreg.predict(X_val) # predict
Y_pred_train = logreg.predict(X_train) # predict
acc_log_val = sum(Y_pred == Y_val)/len(Y_val)*100
acc_log_train = sum(Y_pred_train == Y_train)/len(Y_train)*100
print('Logistic Regression accuracy test:', str(round(acc_log_val,2)),'%')
print('Logistic Regression accuracy train:', str(round(acc_log_train,2)),'%')

Logistic Regression accuracy test: 16.78 %
Logistic Regression accuracy train: 65.32 %


In [52]:
# Support Vector Machines Classifier (non-linear kernel)
svc = SVC()
svc.fit(X_train, Y_train)
Y_pred = svc.predict(X_val) # predict
Y_pred_train = svc.predict(X_train) # predict
acc_svc_val = sum(Y_pred == Y_val)/len(Y_val)*100
acc_svc_train = sum(Y_pred_train == Y_train)/len(Y_train)*100
print('SVC accuracy test:', str(round(acc_svc_val,2)),'%')
print('SVC accuracy train:', str(round(acc_svc_train,2)),'%')

SVC accuracy test: 8.74 %
SVC accuracy train: 59.72 %


In [53]:
perceptron = Perceptron()
perceptron.fit(X_train, Y_train)
Y_pred = perceptron.predict(X_val) # predict
Y_pred_train = perceptron.predict(X_train) # predict
acc_perceptron_val = sum(Y_pred == Y_val)/len(Y_val)*100
acc_perceptron_train = sum(Y_pred_train == Y_train)/len(Y_train)*100
print('Perceptron accuracy test:', str(round(acc_perceptron_val,2)),'%')
print('Perceptron accuracy train:', str(round(acc_perceptron_train,2)),'%')

Perceptron accuracy test: 10.49 %
Perceptron accuracy train: 56.65 %


In [54]:
knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(X_train, Y_train)
Y_pred = knn.predict(X_val) # predict
Y_pred_train = knn.predict(X_train) # predict
acc_knn_val = sum(Y_pred == Y_val)/len(Y_val)*100
acc_knn_train = sum(Y_pred_train == Y_train)/len(Y_train)*100
print('knn accuracy test:', str(round(acc_knn_val,2)),'%')
print('knn accuracy train:', str(round(acc_knn_train,2)),'%')

knn accuracy test: 20.63 %
knn accuracy train: 82.57 %


In [55]:
gradboost = xgb.XGBClassifier(n_estimators=1000)
gradboost.fit(X_train, Y_train)
Y_pred = gradboost.predict(X_val) # predict
Y_pred_train = gradboost.predict(X_train) # predict
acc_gradboost_val = sum(Y_pred == Y_val)/len(Y_val)*100
acc_gradboost_train = sum(Y_pred_train == Y_train)/len(Y_train)*100
print('gradboost accuracy test:', str(round(acc_gradboost_val,2)),'%')
print('gradboost accuracy train:', str(round(acc_gradboost_train,2)),'%')

gradboost accuracy test: 29.72 %
gradboost accuracy train: 97.46 %


In [56]:
random_forest = RandomForestClassifier(n_estimators=1000)
random_forest.fit(X_train, Y_train)
Y_pred = random_forest.predict(X_val) # predict
Y_pred_train = random_forest.predict(X_train) # predict
acc_random_forest_val = sum(Y_pred == Y_val)/len(Y_val)*100
acc_random_forest_train = sum(Y_pred_train == Y_train)/len(Y_train)*100
print('gradboost accuracy test:', str(round(acc_random_forest_val,2)),'%')
print('gradboost accuracy train:', str(round(acc_random_forest_train,2)),'%')

gradboost accuracy test: 32.17 %
gradboost accuracy train: 99.47 %


In [47]:
h_df = pd.read_csv('Hainan_RawBiogas_Categorical.csv')

In [48]:
h_df.head()

,Year,Pig Manure (t),Cassava (t),Fish waste water (t),Kitchen food waste (t),Municipal fecal residue (t),Tea waste (t),Chicken litter (t),Bagasse feed (t),Alcohol waste (t),Chinese medicine waste (t),Energy grass (t),Banana fruit shafts (t),Lemon waste (t),Percolate (t),Other waste (t),Raw Biogas Produced (m3)
0,2014,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5154.0
1,2014,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3426.0
2,2014,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5885.0
3,2014,14.9,0.0,0.0,0.0,0.0,0,0.0,13.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3727.0
4,2014,0.0,0.0,0.0,0.0,0.0,0,0.0,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4072.0


In [49]:
X = h_df.filter( ['Pig Manure (t)', 'Cassava (t)'], axis = 1)